### imports

In [ ]:
import time
import hrr_persistant_memory as hrr
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from collections import Counter
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

### maze enviornment

In [ ]:
# maze enviornment
#  l    42    r
# rg    32    gg
# 20 21 22 23 24
#      ]12[ 
#  0  1  2  3  4
#      - 8
#      -18

def maze(state):
    if state == -18:
        return ['up']
    if state == -8:
        return ['up', 'down']
    if state == 0:
        return ['right']
    if state == 1:
        return ['right', 'left']
    if state == 2:
        return ['right', 'up', 'left', 'down']
    if state == 3:
        return ['right', 'left']
    if state == 4:
        return ['left']
    if state == 12:
        return ['up', 'down']
    if state == 20:
        return ['right']
    if state == 21:
        return ['right', 'left']
    if state == 22:
        return ['right', 'up', 'left', 'down']
    if state == 23:
        return ['right', 'left']
    if state == 24:
        return ['left']
    if state == 32:
        return ['up', 'down']
    if state == 42:
        return ['down']

### hyperparameters

In [ ]:
bias = 1.0
reward = 1.0

gamma = 0.5
alpha = 0.1
lambd = 0.0
epsilon = 0.0

states = 12
episodes = 1 #10000

hrr_size = 1024
non_goal_states = 0

# drop in states
drop_in_states = [0]
gates = ['open', 'closed']

In [ ]:
def encode_gate(state, signal, w_m, gate_action):
    string = 'state_'  + str(state) + "*" + \
             'signal_' + str(signal) + "*" + \
             'w_m_' + str(w_m) + "*" + \
             'gate_action_' + str(gate_action)
    encoded = np.array([memory.encode(string)])
    return(encoded)

### gate functions

In [ ]:
# input gate

def q_input_gate(state, signal, w_m):
    max_value = -99999
    
    if np.random.random() < epsilon:
        gate = np.random.choice(gates)
        hrr = encode_gate(state, signal, w_m, gate)
        value = model_input_gate.predict(hrr)[0,0] + bias
        return value, hrr, gate
    else:
        for gate_action in gates:
            hrr = encode_gate(state, signal, w_m, gate_action)
            value = model_input_gate.predict(hrr)[0,0] + bias
            if value > max_value:
                max_value = value
                max_hrr = hrr
                max_gate = gate_action
        return  max_value, max_hrr, max_gate

In [ ]:
# output gate

def q_output_gate(state, signal, w_m):
    max_value = -99999
    
    if np.random.random() < epsilon:
        gate = np.random.choice(gates)
        hrr = encode_gate(state, signal, w_m, gate)
        value = model_output_gate.predict(hrr)[0,0] + bias
        return value, hrr, gate
    else:
        for gate_action in gates:
            hrr = encode_gate(state, signal, w_m, gate_action)
            value = model_output_gate.predict(hrr)[0,0] + bias
            if value > max_value:
                max_value = value
                max_hrr = hrr
                max_gate = gate_action
        return  max_value, max_hrr, max_gate

In [ ]:
# move gate

def encode_move(state, signal, w_m, phyical_action):
    string = 'state_'  + str(state) + "*" + \
             'signal_' + str(signal) + "*" + \
             'w_m_' + str(w_m) + "*" + \
             'phyical_action_' + str(phyical_action)
    encoded = np.array([memory.encode(string)])
    return(encoded)

def q_move(state, signal, w_m):
    max_value = -99999
    if np.random.random() < epsilon:
        phyical_move = np.random.choice(actions)
        hrr = encode_move(state, signal, w_m, phyical_move)
        value = model_move.predict(hrr)[0,0] + bias
        return value, hrr, phyical_move  
    else:
        for phyical_action in actions:
            hrr = encode_move(state, signal, w_m, phyical_action)
            value = model_move.predict(hrr)[0,0] + bias
            if value > max_value:
                max_value = value
                max_hrr = hrr
                max_action = phyical_action
        return  max_value, max_hrr, max_action

def q_move_max(state, signal, w_m):
    max_value = -99999
#     print(state, actions)
    for phyical_action in actions:    
        hrr = encode_move(state, signal, w_m, phyical_action)
        value = model_move.predict(hrr)[0,0] + bias
        if value > max_value:
            max_value = value
            max_hrr = hrr
            max_action = phyical_action
    return  max_value, max_hrr, max_action

### load the memory created when the code was first run

In [ ]:
memory = hrr.LTM(prefix="my_hrrs", N=hrr_size, normalized=True)

### load the models created when the code was first run

In [ ]:
model_input_gate = keras.models.load_model("model_input_gate.h5")
model_output_gate = keras.models.load_model("model_output_gate.h5")
model_move = keras.models.load_model("model_move.h5")

### run model on each color to see what policy the agent learned
#### red below

In [ ]:
current_state = 0       
current_signal = 'red'

if current_signal == 'red':
    goal_state = 20
if current_signal == 'green':
    goal_state = 24
if current_signal == 'purple':
    goal_state = 42

# clear out working memory amd move content
current_w_m = ''
recall = 'not_remembering'

for step in range(states):
    print(step, current_state)
    # set for same start
    if current_state == goal_state:
        if color_start == 0:
            sub_optimal_steps.append(step - 6)
        break
    else:
        if step == (states-1):
            sub_optimal_steps.append(states)

    actions = maze(current_state)

    if current_signal != '':

        # need to make this color
        back_up = current_signal
        current_signal = 'color'

        gate_value, \
        gate_hrr, \
        gate_action = q_input_gate(current_state, current_signal, recall)

        current_signal = back_up

        move_value, \
        move_hrr, \
        move_action = q_move(current_state, current_signal, recall)

        previous_state = current_state
        previous_gate_hrr = gate_hrr
        previous_move_value = move_value
        previous_move_hrr = move_hrr

        if gate_action == 'open':
            current_w_m = current_signal
            recall = 'remembering'

        current_signal = ''            

        # make move to next state      
        if (move_action == "left"):
            current_state = (current_state-1)
        if (move_action == "right"):
            current_state = (current_state+1)
        if (move_action == "up"):
            current_state = (current_state+10)
        if (move_action == "down"):
            current_state = (current_state-10)

        # now current_state is the next state so check new actions
        actions = maze(current_state)

        if current_state == goal_state:
            target = reward
        else:
            if recall == 'remembering':
                _, _, gate_action = q_output_gate(current_state, current_signal, recall)
            else:
                gate_action = 'closed'

            if gate_action == 'open':
                save_recall = recall
                recall = current_w_m
                move_value, _, _ = q_move_max(current_state, current_signal, recall)
                recall = save_recall
            else:
                move_value, _, _ = q_move_max(current_state, current_signal, recall)

            target = non_goal_states + gamma * move_value

    else:                
        gate_value, \
        gate_hrr, \
        gate_action = q_output_gate(current_state, current_signal, recall)

        if gate_action == 'open':
            recall = current_w_m
            current_w_m = ''
            move_value, \
            move_hrr, \
            move_action = q_move(current_state, current_signal, recall)
            recall = 'memory_used'
        else:
            move_value, \
            move_hrr, \
            move_action = q_move(current_state, current_signal, recall)

        previous_state = current_state
        previous_move_value = move_value
        previous_move_hrr = move_hrr
        previous_gate_hrr = gate_hrr

        if (move_action == "left"):
            current_state = (current_state-1)
        if (move_action == "right"):
            current_state = (current_state+1)
        if (move_action == "up"):
            current_state = (current_state+10)
        if (move_action == "down"):
            current_state = (current_state-10)

        actions = maze(current_state)

        if current_state == goal_state:
            target = reward
        else:
            if recall == 'remembering':
                _, _, gate_action = q_output_gate(current_state, current_signal, recall)
            else:
                gate_action = 'closed'

            if gate_action == 'open':
                save_recall = recall
                recall = current_w_m
                move_value, _, _ = q_move_max(current_state, current_signal, recall)
                recall = save_recall
            else:
                move_value, _, _ = q_move_max(current_state, current_signal, recall)

            target = non_goal_states + gamma * move_value

##### we can see red did not behave well








#### green below

In [ ]:
current_state = 0       
current_signal = 'green'

if current_signal == 'red':
    goal_state = 20
if current_signal == 'green':
    goal_state = 24
if current_signal == 'purple':
    goal_state = 42

# clear out working memory amd move content
current_w_m = ''
recall = 'not_remembering'

for step in range(states):
    print(step, current_state)
    # set for same start
    if current_state == goal_state:
        if color_start == 0:
            sub_optimal_steps.append(step - 6)
        break
    else:
        if step == (states-1):
            sub_optimal_steps.append(states)

    actions = maze(current_state)

    if current_signal != '':

        # need to make this color
        back_up = current_signal
        current_signal = 'color'

        gate_value, \
        gate_hrr, \
        gate_action = q_input_gate(current_state, current_signal, recall)

        current_signal = back_up

        move_value, \
        move_hrr, \
        move_action = q_move(current_state, current_signal, recall)

        previous_state = current_state
        previous_gate_hrr = gate_hrr
        previous_move_value = move_value
        previous_move_hrr = move_hrr

        if gate_action == 'open':
            current_w_m = current_signal
            recall = 'remembering'

        current_signal = ''            

        # make move to next state      
        if (move_action == "left"):
            current_state = (current_state-1)
        if (move_action == "right"):
            current_state = (current_state+1)
        if (move_action == "up"):
            current_state = (current_state+10)
        if (move_action == "down"):
            current_state = (current_state-10)

        # now current_state is the next state so check new actions
        actions = maze(current_state)

        if current_state == goal_state:
            target = reward
        else:
            if recall == 'remembering':
                _, _, gate_action = q_output_gate(current_state, current_signal, recall)
            else:
                gate_action = 'closed'

            if gate_action == 'open':
                save_recall = recall
                recall = current_w_m
                move_value, _, _ = q_move_max(current_state, current_signal, recall)
                recall = save_recall
            else:
                move_value, _, _ = q_move_max(current_state, current_signal, recall)

            target = non_goal_states + gamma * move_value

    else:                
        gate_value, \
        gate_hrr, \
        gate_action = q_output_gate(current_state, current_signal, recall)

        if gate_action == 'open':
            recall = current_w_m
            current_w_m = ''
            move_value, \
            move_hrr, \
            move_action = q_move(current_state, current_signal, recall)
            recall = 'memory_used'
        else:
            move_value, \
            move_hrr, \
            move_action = q_move(current_state, current_signal, recall)

        previous_state = current_state
        previous_move_value = move_value
        previous_move_hrr = move_hrr
        previous_gate_hrr = gate_hrr

        if (move_action == "left"):
            current_state = (current_state-1)
        if (move_action == "right"):
            current_state = (current_state+1)
        if (move_action == "up"):
            current_state = (current_state+10)
        if (move_action == "down"):
            current_state = (current_state-10)

        actions = maze(current_state)

        if current_state == goal_state:
            target = reward
        else:
            if recall == 'remembering':
                _, _, gate_action = q_output_gate(current_state, current_signal, recall)
            else:
                gate_action = 'closed'

            if gate_action == 'open':
                save_recall = recall
                recall = current_w_m
                move_value, _, _ = q_move_max(current_state, current_signal, recall)
                recall = save_recall
            else:
                move_value, _, _ = q_move_max(current_state, current_signal, recall)

            target = non_goal_states + gamma * move_value

##### we can see green is sub-optimal but still solving the task








#### purple below

In [ ]:
current_state = 0       
current_signal = 'purple'

if current_signal == 'red':
    goal_state = 20
if current_signal == 'green':
    goal_state = 24
if current_signal == 'purple':
    goal_state = 42

# clear out working memory amd move content
current_w_m = ''
recall = 'not_remembering'

for step in range(states):
    print(step, current_state)
    # set for same start
    if current_state == goal_state:
        if color_start == 0:
            sub_optimal_steps.append(step - 6)
        break
    else:
        if step == (states-1):
            sub_optimal_steps.append(states)

    actions = maze(current_state)

    if current_signal != '':

        # need to make this color
        back_up = current_signal
        current_signal = 'color'

        gate_value, \
        gate_hrr, \
        gate_action = q_input_gate(current_state, current_signal, recall)

        current_signal = back_up

        move_value, \
        move_hrr, \
        move_action = q_move(current_state, current_signal, recall)

        previous_state = current_state
        previous_gate_hrr = gate_hrr
        previous_move_value = move_value
        previous_move_hrr = move_hrr

        if gate_action == 'open':
            current_w_m = current_signal
            recall = 'remembering'

        current_signal = ''            

        # make move to next state      
        if (move_action == "left"):
            current_state = (current_state-1)
        if (move_action == "right"):
            current_state = (current_state+1)
        if (move_action == "up"):
            current_state = (current_state+10)
        if (move_action == "down"):
            current_state = (current_state-10)

        # now current_state is the next state so check new actions
        actions = maze(current_state)

        if current_state == goal_state:
            target = reward
        else:
            if recall == 'remembering':
                _, _, gate_action = q_output_gate(current_state, current_signal, recall)
            else:
                gate_action = 'closed'

            if gate_action == 'open':
                save_recall = recall
                recall = current_w_m
                move_value, _, _ = q_move_max(current_state, current_signal, recall)
                recall = save_recall
            else:
                move_value, _, _ = q_move_max(current_state, current_signal, recall)

            target = non_goal_states + gamma * move_value

    else:                
        gate_value, \
        gate_hrr, \
        gate_action = q_output_gate(current_state, current_signal, recall)

        if gate_action == 'open':
            recall = current_w_m
            current_w_m = ''
            move_value, \
            move_hrr, \
            move_action = q_move(current_state, current_signal, recall)
            recall = 'memory_used'
        else:
            move_value, \
            move_hrr, \
            move_action = q_move(current_state, current_signal, recall)

        previous_state = current_state
        previous_move_value = move_value
        previous_move_hrr = move_hrr
        previous_gate_hrr = gate_hrr

        if (move_action == "left"):
            current_state = (current_state-1)
        if (move_action == "right"):
            current_state = (current_state+1)
        if (move_action == "up"):
            current_state = (current_state+10)
        if (move_action == "down"):
            current_state = (current_state-10)

        actions = maze(current_state)

        if current_state == goal_state:
            target = reward
        else:
            if recall == 'remembering':
                _, _, gate_action = q_output_gate(current_state, current_signal, recall)
            else:
                gate_action = 'closed'

            if gate_action == 'open':
                save_recall = recall
                recall = current_w_m
                move_value, _, _ = q_move_max(current_state, current_signal, recall)
                recall = save_recall
            else:
                move_value, _, _ = q_move_max(current_state, current_signal, recall)

            target = non_goal_states + gamma * move_value

## purple did great!

In [ ]:
ep_count = np.load('ep_count.npy', allow_pickle=True)
final_arr = np.load('final_arr.npy', allow_pickle=True)

In [ ]:
ep_count

### plots to make advisor happy

In [ ]:
plt.plot(final_arr)
plt.grid(True)
plt.xlabel('Episodes')
plt.ylabel('Sub-optimal Steps')
# plt.savefig('red_green_purple.png')
plt.show()

### clean up memory

In [ ]:
memory = None